# Rapid Adiabatic Passage Simulations for $\text{CO}$

In [ ]:
# importing some paraphernelia 
import numpy as np
import matplotlib as mpl
mpl.rc('text', usetex=True)
mpl.rc('font', family='sans-serif', serif='computer modern')
import matplotlib.pyplot as plt

Also, the lifetime of an excited state $\tau$ and the corresponding einstein coefficient of the transition $A_{1\rightarrow 0}$ are related as,
$$
A_{1\rightarrow 0} = \frac{1}{\tau}.
$$
where $\tau$ is an experimentally, measurably quantity.

We can transition dipole moment from $A_{1\rightarrow 0}$ as,

$$
A_{1\rightarrow 0} = \frac{16\pi^3\bar{\nu}^3}{3\varepsilon_{0}h}\mu_{10}^{2}
$$

(For reference see, the textbook by Peter Bernath.)


In [ ]:
from scipy.constants import epsilon_0, mu_0, h

def calc_dipole(A, nu_bar):
    """Calculates transition dipole matrix element mu_10 (C-m) for a transition with einstein coefficient A (s^{-1}) and, correspdoning wavenumber nu_bar (cm^-1)
    """
    const = 16*np.pi**3/(3*epsilon_0*h)    
    nu_bar = 100*nu_bar
    mu_10 = np.sqrt(A/(const*nu_bar**3))
    return mu_10

In [ ]:
A10 = 50.61
dipole = calc_dipole(A10, nu_bar=2142.719943)
print("Dipole moment of the CO molecule : %8.6E C-m"%dipole)
print('(OR) %6.2f D'%(dipole/(3.33564*1E-30)))

In [ ]:
run_params = {
    # Laser parameters
    'wl_nm' : 4666, # wavelength [nm]
    'r' : 0.2,      # radius of the laser beam [cm]   
    'U0' : 0.0,     # real part of coherences
    'V0' : 0.0,     # imag part of coherences
    'W0' : -1.0,    # population difference (N_excited - N_ground)
    'dipole' : dipole, # dipole moment in [Cm]
}

In [ ]:
from helpers.rapmb import *

In [6]:
# Monte carlo test for Fig.6
# f = 2.5
from helpers.rapmb import *
run_params['N_t'] = 40
run_params['N_samples'] = 1000 
run_params['divergence'] = 0.250
run_params['delta_v'] = 5 
run_params['v'] = 1200  # m/s
run_params['P'] = 0.020 # 20mW laser power
Nz = 25
z = np.linspace(0.1, 50.0, Nz)
focal_length = [15.0]# 18.0#, 20.0, 25.4]
data = {}
for f in focal_length:
    run_params['f'] = f
    n1 = np.empty(Nz, dtype=np.float64)
    for i in range(Nz):
        run_params['zi'] = z[i]
        n1[i] = monte_carlo(run_params)
    data[f] = n1
print("Done Simulations!")

In [ ]:
data['z'] = z
np.save('./data/co_bloch_trajectories/p_20e-3_v_12e2_fdata.npy', data)


In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
ax.plot(z, data[15.0], label=15.0)
ax.plot(z, data[18.0], label=18.0)
ax.plot(z, data[20.0], label=20.0)
ax.plot(z, data[25.4], label=25.4)
# ax.plot(z, data[40.0], label=40.0)
ax.set_xlim(0, )
ax.set_ylim(0, 1.0)
ax.set_xlabel(r'$z$ [cm]', size=14)
ax.set_ylabel(r'$N_{1}$', size=14)
ax.legend(bbox_to_anchor=[.5,0.5], frameon=False, title=r'$f$ [cm]')
fig.tight_layout()